# Validation of the classification based on the WFsim #

2019/09/28

Authors:
 - Clark, Michael <clark632@purdue.edu>
 - Angevaare, Joran <j.angevaare@nikhef.nl>
 
**Updates:**

2019/11/14

## This notebook #
 
 
Possible extensions:
 - Add afterpulse boolian to the 'truth' info
 - Do the same for the other detector types

In [1]:
import strax
import straxen

In [2]:
import wfsim

In [3]:
import os

In [4]:
import pandas as pd

In [5]:
pd.set_option('display.max_rows', 50)

We include ``recarray_tools.py`` here that is used to add columns and do things with structured arrays. 
Taken from:

    https://github.com/XENON1T/XeAnalysisScripts/tree/master/PeakFinderTest

In [6]:
from peak_classification.recarray_tools import *

In [7]:
from peak_classification.wfsim_utils import *

In [8]:
from peak_classification.match_peaks import *

Initize the wavefrom simulator

In [9]:
instructions_csv = "./test_uni.csv"
instructions = dict(event_rate = 100 , chunk_size=10, nchunk=5)
inst_to_csv(instructions, instructions_csv)

In [10]:
st = strax.Context(
    register=[wfsim.RawRecordsFromFax],
    config=dict(fax_file=instructions_csv),
    **straxen.contexts.common_opts)

In [11]:
class CustomPeakClassification(strax.Plugin):
    # Name of the data type this plugin provides
    provides = 'peak_classification'
    
    # Data types this plugin requires. Note we don't specify
    # what plugins should produce them: maybe the default PeakBasics
    # has been replaced by another AdvancedExpertBlabla plugin?
    depends_on = ('peak_basics')
    
    # Numpy datatype of the output 
    dtype = straxen.PeakClassification.dtype
    
    # Version of the plugin. Increment this if you change the algorithm.
    __version__ = '0.0.1'
    
    result = {}
    def compute(self, peaks):
        result = np.zeros(len(peaks), dtype=self.dtype)
        masks1 = (peaks['n_channels']>2) & (peaks['range_50p_area'] < 100)
        
        result['type'][masks1] = 1
        masks2 = (peaks['n_channels']>2) & (peaks['area'] > 20) & (peaks['range_50p_area'] > 100)
        result['type'][masks2] = 2

        return result

In [12]:
# Just some id from post-SR1, so the corrections work
run_id = '180519_1902'

In [ ]:
if check_for_strax_data():
    !rm -r strax_data/
    print('deleted data')

peaks = st.make(run_id, 'raw_records')

Data found in 'strax_data', press [y] to remove and create new data
y
deleted data


Simulating Raw Records:   6%|▋         | 634/10000 [00:36<23:31,  6.64it/s]

In [ ]:
truth = st.get_array(run_id, 'truth')
data_default = st.get_array(run_id, ['peak_basics','peak_classification'])
data_custom = st.get_array(run_id, ['peak_basics','peak_classification'],
                          register=CustomPeakClassification)

This is to compensate for the fact that we dont have event numbers (Binning in time to group peaks)

In [ ]:
timing_grid = get_timing_grid(instructions)

In [ ]:
### Proxy for event number
truth = append_fields(truth, 'merge_index',np.digitize(truth['t'], timing_grid))
data_default = append_fields(data_default, 'merge_index', 
                             np.digitize(data_strax['time'], timing_grid))
data_custom = append_fields(data_custom, 'merge_index', 
                            np.digitize(data_strax['time'], timing_grid))

**There is a bug that the types are listed here as strings, where in strax they are integers**
The code here is to change that such that we can compare them directly

Proxy for ``left`` and ``right`` (as in ``PAX``) sides of peak in truth.

In [ ]:
### Proxy for left and right of peak
truth = append_fields(truth, 
                      ('time','endtime'), 
                      (truth['t_first_photon'],
                       truth['t_last_photon']))

Here in match_peaks.py, written by Jelle, to compare two sets of peaks

Call with (truth, data)

In [ ]:
thruth_vs_default, default_vs_truth = match_peaks(truth,data_default)
thruth_vs_custom, custom_vs_truth = match_peaks(truth,data_custom)

Below is the output of match_peaks for the truth data.  
  - For each peak, **outcome** shows whether the peak was found, missed, merged, split up, or misidentified in the output of strax for the simulated data
  - **matched_to** shows which peak (peak_id in the other array) it was matched with, or the biggest peak it was matched with 

<img src='toptruthmatches.png'>
  
Below is the corresponding match_index in the simulated data
<img src='topdatamatch.png'>
  
You can see the splitting of the true s2 into an s1 and an s2

In [ ]:
headers = ['merge_index','type','time','n_photon','endtime','matched_to','outcome']
pd.DataFrame.from_records(thruth_vs_default[headers])


In [ ]:
# headers = ['merge_index','type','time','area','endtime','matched_to','outcome']
# pd.DataFrame.from_records(datamatched[headers]).head(10)
# #pd.DataFrame.from_records(truthmatched[['merge_index','type','time','area','endtime','matched_to','outcome']])


In [ ]:
# headers = ['merge_index','type','time','n_photon','endtime','matched_to','outcome']
# pd.DataFrame.from_records(truthmatched[truthmatched['outcome'] == b'found'][headers])

## Plotting the results ##
The plots below show the fraction of several of the ``dtypes`` of the ``truth`` or the ``data``. These fractions show how many of the ``peaks`` were found correctly.

The default ``strax`` options, first:

In [ ]:
plot_peak_matching_histogram(thruth_vs_default,'type',bins=[-0.5,0.5,1.5,2.5])
plt.xlabel('Peak Type')
plt.show()

The custom options as introduced above:

In [ ]:
plot_peak_matching_histogram(thruth_vs_custom,'type',bins=[-0.5,0.5,1.5,2.5])
plt.xlabel('Peak Type')
plt.show()